<a href="https://colab.research.google.com/github/Lee-HT/NLP/blob/main/selements_of_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.3 MB/s 
     |████████████████████████████████| 7.6 MB 43.5 MB/s 
     |████████████████████████████████| 163 kB 22.2 MB/s 


In [2]:
import zipfile
import shutil
import os

root_dir = '/content'
data_dir = '/content/tweets'

if os.path.exists(data_dir):
  shutil.rmtree(data_dir)

with zipfile.ZipFile(os.path.join(root_dir,'drive/MyDrive/data','sentiments_tweets.zip'),'r') as file:
  file.extractall(data_dir)

In [3]:
import pandas as pd

DataFrame = pd.read_csv(os.path.join(data_dir,'dataset.csv'))

print(DataFrame.head(10),'\n')

df_group = DataFrame.groupby('Language')

df_en = df_group.get_group('en')

print(df_en.head(10),'\n')
print(len(df_en))

                                                Text Language        Label
0  @Charlie_Corley @Kristine1G @amyklobuchar @Sty...       en    litigious
1  #BadBunny: Como dos gotas de agua: Joven se di...       es     negative
2  https://t.co/YJNiO0p1JV Flagstar Bank disclose...       en    litigious
3  Rwanda is set to host the headquarters of Unit...       en     positive
4  OOPS. I typed her name incorrectly (today’s br...       en    litigious
5  It sucks for me since I'm focused on the natur...       en     negative
6  @en_font Treballar a l’obra a partir dels 19 a...       ca     negative
7  @ShawnTarloff @itsmieu you can also relate thi...       en  uncertainty
8  Social Security. Constant political crises dis...       en     negative
9  @FilmThePoliceLA A broken rib can puncture a l...       en     negative 

                                                 Text Language        Label
0   @Charlie_Corley @Kristine1G @amyklobuchar @Sty...       en    litigious
2   https://t.co/YJNi

In [4]:
from nltk.corpus import stopwords
import re

def clear(text):
  com = re.compile('[^\w\d ]')
  text = com.sub('',text)
  
  return text.lower()

df_clear = [clear(t) for t in df_en['Text']]
print(df_clear[:5])

['charlie_corley kristine1g amyklobuchar stylewriternyc testimony is not evidence in a court of law state or federal must stand up to cross examination', 'httpstcoyjnio0p1jv flagstar bank discloses a data breach that impacted 15million individuals cybersecurity', 'rwanda is set to host the headquarters of united nations development programmes undp new innovation financing facility find out more httpstcobznktc8pl0sovereigngroup sovereignafrica undp rwanda', 'oops i typed her name incorrectly todays brave witness cassidyhutchinson 6 premeditatedviolence 7 probaby witnesstamperig dont forget markmeadows requesting a pardon trumpknew trumpcapitolcoupattack capitolcoupattempt neverforgetjanuary6th neverforgetjan6th httpstcooexm3hdwnz', 'it sucks for me since im focused on the nature aspect of things but the environmental problems we have all have their roots in social economic amp political issues simply recycling or using renewable energy is like fixing the top floor of a building with an 

In [5]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

token_dict = tokenizer.get_vocab()
print('len tokens= ',len(token_dict))

mlen = max([len(_.split(' ')) for _ in df_clear])
print('df_len= ',mlen)

#data_len = 512

df_encode = [tokenizer.encode(data,max_length=mlen,truncation=True) for data in df_clear]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

len tokens=  30522
df_len=  161


In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_set = pad_sequences(df_encode,maxlen=mlen,padding='pre')

print(train_set[0])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0   101  4918  1035  2522 12866 25130  2063  2487
  2290  6864  2243  4135 25987  2906  2806 15994  4890  2278 10896  2003
  2025  3350  1999  1037  2457  1997  2375  2110  2030  2976  2442  3233
  2039  2000  2892  7749   102]


In [7]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()

df_lb = df_en['Label']
print(df_lb.head(5),'\n')
print('Value_counts\n' , df_lb.value_counts() , '\n')

Label = LE.fit_transform(df_lb)
classes = LE.classes_
print(classes)

0    litigious
2    litigious
3     positive
4    litigious
5     negative
Name: Label, dtype: object 

Value_counts
 positive       248516
negative       244146
uncertainty    198586
litigious      180062
Name: Label, dtype: int64 

['litigious' 'negative' 'positive' 'uncertainty']


In [8]:
from sklearn.model_selection import train_test_split

train_set,train_test,Label,Label_test = train_test_split(train_set,Label,test_size = 0.1,stratify = Label)

In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Activation

input_shape = mlen
total_word = len(token_dict) + 1

Model = Sequential([
    Embedding(total_word,30,input_length=input_shape),
    LSTM(100),
    Dense(len(classes)),Activation('softmax')
])

In [10]:
Model.compile(loss ='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),metrics=['accuracy'])

Model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 161, 30)           915690    
                                                                 
 lstm (LSTM)                 (None, 100)               52400     
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
 activation (Activation)     (None, 4)                 0         
                                                                 
Total params: 968,494
Trainable params: 968,494
Non-trainable params: 0
_________________________________________________________________


In [11]:
Model.fit(train_set,Label,epochs = 4)

Epoch 1/4
24506/24506 [==============================] - 262s 10ms/step - loss: 0.2160 - accuracy: 0.9245
Epoch 2/4
24506/24506 [==============================] - 253s 10ms/step - loss: 0.0735 - accuracy: 0.9727
Epoch 3/4
24506/24506 [==============================] - 256s 10ms/step - loss: 0.0608 - accuracy: 0.9760
Epoch 4/4
24506/24506 [==============================] - 269s 11ms/step - loss: 0.0553 - accuracy: 0.9781


In [12]:
res = Model.evaluate(train_test,Label_test)
print(res)

2723/2723 [==============================] - 17s 6ms/step - loss: 0.0663 - accuracy: 0.9740
[0.06634043902158737, 0.9739702343940735]


In [13]:
test_str = "have a good day"
test_enc = tokenizer.encode(test_str,max_length=mlen,truncation = True)
test_pad = pad_sequences([test_enc],maxlen=mlen,padding='pre')

pre = Model.predict(test_pad)

print(pre)
print(LE.inverse_transform([np.array(pre).argmax()]))

1/1 [==============================] - 0s 406ms/step
[[4.0770258e-04 1.7582854e-03 9.9745411e-01 3.7994995e-04]]
['positive']
